# GPU Test

## GPU vs CPU vs GPU seed only vs ref

In [11]:
import time
import numpy as np
import random

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# ============================================================
# GPU/CPU energy functions (same definition, different backend)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    """CPU energy using NumPy (matches your calculate_energy_vectorized)."""
    N = len(s)
    e = 0
    # keep integer math; cast at end
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


def energy_cupy(s_cp) -> float:
    """
    GPU energy using CuPy.
    Input: cupy array of shape (N,), dtype int8/int16/int32 recommended.
    Output: Python float (synchronized).
    """
    N = int(s_cp.shape[0])
    e = cp.int64(0)
    for k in range(1, N):
        Ck = cp.sum(s_cp[:N-k] * s_cp[k:])
        e += Ck * Ck
    # synchronize and return host float
    return float(e.get())

def generate_population_gpu_seed_only(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    Use GPU RNG to generate initial population, then transfer to CPU numpy arrays.
    Output population is a list of np.ndarray (dtype int8) suitable for CPU MTS.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot use gpu_seed_only mode.")

    cp.random.seed(seed)

    # Generate 0/1 then map to ±1, store compactly
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}

    # Transfer once to CPU
    pop_cpu = cp.asnumpy(pop_gpu)  # shape (K,N), dtype int8

    # Convert to list of 1D arrays (matches your code structure)
    return [pop_cpu[i].copy() for i in range(K)]

# ============================================================
# Wrappers: adapt your MTS to run with either energy backend
#   (minimal surgical edits: pass energy_fn and array module)
# ============================================================

def flip_inplace(arr, i):
    """In-place flip for numpy/cupy arrays."""
    arr[i] *= -1


def tabu_search_backend(s0, energy_fn, xp, max_iter=None):
    """
    Same logic as your tabu_search, but:
      - uses in-place flips to reduce allocations
      - uses energy_fn backend
      - uses xp (np or cp) for array ops
    Returns: best sequence (same type as input, numpy or cupy array)
    """
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_fn(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)  # tabu list small; keep on CPU

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        # Explore 1-flip neighborhood
        for i in range(N):
            # flip temporarily
            flip_inplace(s, i)
            E_neighbor = energy_fn(s)
            flip_inplace(s, i)  # flip back

            is_tabu = tabu_list[i] > t

            # NOTE: Fix aspiration bug: tabu moves must still compete for "best admissible"
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            # Commit best move
            flip_inplace(s, best_i)

            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_backend(
    N,
    K,
    G_max,
    p_comb=0.9,
    p_mut=None,
    E_target=None,
    use_optimal_target=True,
    stop_at_target=True,
    energy_fn=energy_numpy,
    xp=np,
    init_population=None,   # <---- ADD THIS
):
    if p_mut is None:
        p_mut = 1.0 / N

    if use_optimal_target and (N in LABS_OPTIMAL) and (E_target is None):
        E_target = LABS_OPTIMAL[N]

    # -------------------------
    # Initialization (new)
    # -------------------------
    if init_population is not None:
        # ⚠️ assume caller provides list length K of arrays length N, already on correct device
        population = init_population
    else:
        if xp is np:
            population = [np.random.choice([-1, 1], size=N).astype(np.int8) for _ in range(K)]
        else:
            population = [cp.asarray(np.random.choice([-1, 1], size=N).astype(np.int8)) for _ in range(K)]

    # initial best
    s_best = population[0].copy()
    E_best = energy_fn(s_best)

    for ind in population[1:]:
        E = energy_fn(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    for _ in range(G_max):
        if stop_at_target and (E_target is not None) and (E_best <= E_target):
            break

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = xp.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        child = tabu_search_backend(child, energy_fn=energy_fn, xp=xp)

        E_child = energy_fn(child)
        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

    return float(E_best), s_best


# ============================================================
# Benchmark across N: CPU vs GPU
# ============================================================

def benchmark_cpu_vs_gpu(
    N_list,
    K=100,
    G_max=500,
    trials=1,
    stop_at_target=False,   # IMPORTANT for fair runtime comparisons
    seed=42,
):
    """
    Returns a list of dict rows (easy to convert to pandas).
    Measures wall-clock runtime for full MTS run.
    """
    rows = []

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, None)

        for t in range(trials):
            # ---- CPU run
            np.random.seed(seed + 1000*t + N)
            random.seed(seed + 1000*t + N)

            t0 = time.perf_counter()
            E_cpu, _ = memetic_tabu_search_backend(
                N=N, K=K, G_max=G_max,
                use_optimal_target=True,
                stop_at_target=stop_at_target,
                energy_fn=energy_numpy,
                xp=np,
            )
            t1 = time.perf_counter()

            row = {
                "N": N,
                "trial": t,
                "reference_energy": ref,
                "cpu_energy": E_cpu,
                "cpu_runtime_s": t1 - t0,
            }

            # ---- GPU run (if available)
            if HAS_CUPY:
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                # Warm-up (reduces first-call overhead)
                _ = cp.asarray(np.array([1, -1], dtype=np.int8))
                cp.cuda.Stream.null.synchronize()

                g0 = time.perf_counter()
                E_gpu, _ = memetic_tabu_search_backend(
                    N=N, K=K, G_max=G_max,
                    use_optimal_target=True,
                    stop_at_target=stop_at_target,
                    energy_fn=energy_cupy,
                    xp=cp,
                )
                cp.cuda.Stream.null.synchronize()
                g1 = time.perf_counter()

                row.update({
                    "gpu_energy": E_gpu,
                    "gpu_runtime_s": g1 - g0,
                    "cpu_gpu_energy_match": (abs(E_cpu - E_gpu) < 1e-9),
                })
            else:
                row.update({
                    "gpu_energy": None,
                    "gpu_runtime_s": None,
                    "cpu_gpu_energy_match": None,
                })
            # ---- GPU seed-only run (GPU for population init only; rest CPU)
            if HAS_CUPY:
                np.random.seed(seed + 3000*t + N)        # CPU RNG for all decisions after init
                random.seed(seed + 3000*t + N)

                # Warm-up GPU once
                _ = cp.asarray(np.array([1, -1], dtype=np.int8))
                cp.cuda.Stream.null.synchronize()

                # GPU generates initial population -> copied to CPU
                pop_seed = generate_population_gpu_seed_only(
                    N=N, K=K, seed=seed + 3000*t + N
                )

                s0 = time.perf_counter()
                E_seedonly, _ = memetic_tabu_search_backend(
                    N=N, K=K, G_max=G_max,
                    use_optimal_target=True,
                    stop_at_target=stop_at_target,
                    energy_fn=energy_numpy,   # CPU energy
                    xp=np,                    # CPU arrays for the algorithm
                    init_population=pop_seed  # <-- only init comes from GPU
                )
                s1 = time.perf_counter()

                row.update({
                    "gpu_seedonly_energy": E_seedonly,
                    "gpu_seedonly_runtime_s": s1 - s0,
                })
            else:
                row.update({
                    "gpu_seedonly_energy": None,
                    "gpu_seedonly_runtime_s": None,
                })

            # gaps vs reference
            if ref is not None and ref != 0:
                row["cpu_gap_%"] = 100.0 * (E_cpu - ref) / ref
                if row["gpu_energy"] is not None:
                    row["gpu_gap_%"] = 100.0 * (row["gpu_energy"] - ref) / ref
                else:
                    row["gpu_gap_%"] = None
            else:
                row["cpu_gap_%"] = None
                row["gpu_gap_%"] = None

            rows.append(row)

    return rows


# Example usage
if __name__ == "__main__":
    N_list = [10, 12, 15]

    rows = benchmark_cpu_vs_gpu(
        N_list,
        K=100,
        G_max=300,
        trials=1,
        stop_at_target=False,
        seed=42
    )

    # Convert results into a clean table
    import pandas as pd

    df = pd.DataFrame(rows)

    # Keep only the most important benchmark columns
    df = df[[
        "N",
        "reference_energy",
        "cpu_energy",
        "cpu_runtime_s",
        "gpu_seedonly_energy",
        "gpu_seedonly_runtime_s",
        "gpu_energy",
        "gpu_runtime_s",
        "cpu_gap_%",
        "gpu_gap_%",
        "cpu_gpu_energy_match"
    ]]



    # Round numeric values for readability
    df["cpu_runtime_s"] = df["cpu_runtime_s"].round(4)
    df["gpu_runtime_s"] = df["gpu_runtime_s"].round(4)
    df["gpu_seedonly_runtime_s"] = df["gpu_seedonly_runtime_s"].round(4)


    df["cpu_gap_%"] = df["cpu_gap_%"].round(2)
    df["gpu_gap_%"] = df["gpu_gap_%"].round(2)

    print("\n" + "="*90)
    print(" CPU vs GPU MTS Benchmark Results (LABS)")
    print("="*90)
    print(df.to_string(index=False))
    print("="*90)



 CPU vs GPU MTS Benchmark Results (LABS)
 N  reference_energy  cpu_energy  cpu_runtime_s  gpu_seedonly_energy  gpu_seedonly_runtime_s  gpu_energy  gpu_runtime_s  cpu_gap_%  gpu_gap_%  cpu_gpu_energy_match
10                13        13.0         1.0020                 13.0                  1.0043        13.0        24.4374        0.0        0.0                  True
12                10        10.0         1.8730                 10.0                  1.8976        10.0        45.0493        0.0        0.0                  True
15                15        15.0         3.6052                 15.0                  3.5829        15.0        82.9974        0.0        0.0                  True


## Higher N test

In [13]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# Your LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# GPU seed-only population generation
# ============================================================

def generate_population_gpu_seed_only(N: int, K: int, seed: int) -> list:
    """
    Use GPU RNG to generate initial population on GPU, then transfer to CPU.
    Returns list[np.ndarray] (dtype int8), length K.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run gpu_seed_only mode.")

    cp.random.seed(seed)

    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}

    pop_cpu = cp.asnumpy(pop_gpu)  # one transfer
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Your MTS pieces (CPU-only execution)
#   - Assumes you have: tabu_search_backend, memetic_tabu_search_backend
#   - Here we include CPU versions only, matching your earlier backend
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_backend_cpu(s0, max_iter=None):
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration fixed: tabu moves still compete for best admissible
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)

            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int = 100,
    G_max: int = 500,
    p_comb: float = 0.9,
    p_mut: float = None,
    stop_at_target: bool = False,
    init_population: list | None = None,
):
    if p_mut is None:
        p_mut = 1.0 / N

    E_target = LABS_OPTIMAL.get(N, None)

    if init_population is not None:
        population = init_population
    else:
        population = [np.random.choice([-1, 1], size=N).astype(np.int8) for _ in range(K)]

    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    for g in range(G_max):
        if stop_at_target and (E_target is not None) and (E_best <= E_target):
            break

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        child = tabu_search_backend_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

    return float(E_best), s_best


# ============================================================
# Crash-resilient logging helpers
# ============================================================

def append_jsonl(path: str, record: dict):
    """Append a JSON record to a .jsonl file (one record per line)."""
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record) + "\n")


def append_csv(path: str, record: dict, header_order: list):
    """Append a row to CSV, creating header if file doesn't exist."""
    new_file = not os.path.exists(path)
    line = ",".join(str(record.get(k, "")) for k in header_order)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header_order) + "\n")
        f.write(line + "\n")


# ============================================================
# Main benchmark (CPU vs GPU-seed-only)
# ============================================================

def run_benchmark_highN(
    N_list=(20, 25, 30),
    K=100,
    G_max=300,
    trials=1,
    seed=42,
    out_prefix="highN_mts_benchmark",
):
    """
    Runs CPU baseline and GPU-seed-only initialization benchmark.
    Logs after every run so partial progress is never lost.
    """
    jsonl_path = out_prefix + ".jsonl"
    csv_path = out_prefix + ".csv"

    columns = [
        "timestamp",
        "N",
        "trial",
        "reference_energy",
        "cpu_energy",
        "cpu_runtime_s",
        "gpu_seedonly_energy",
        "gpu_seedonly_runtime_s",
        "cpu_gap_pct",
        "gpu_seedonly_gap_pct",
        "status_cpu",
        "status_gpu_seedonly",
        "error_cpu",
        "error_gpu_seedonly",
    ]

    print(f"[LOG] Writing incremental results to:\n  {csv_path}\n  {jsonl_path}\n")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, None)
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "reference_energy": ref,
                "cpu_energy": "",
                "cpu_runtime_s": "",
                "gpu_seedonly_energy": "",
                "gpu_seedonly_runtime_s": "",
                "cpu_gap_pct": "",
                "gpu_seedonly_gap_pct": "",
                "status_cpu": "NOT_RUN",
                "status_gpu_seedonly": "NOT_RUN",
                "error_cpu": "",
                "error_gpu_seedonly": "",
            }

            # -------------------------
            # CPU baseline
            # -------------------------
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)
                print("[CPU] starting...")

                t0 = time.perf_counter()
                E_cpu, _ = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    stop_at_target=False,
                    init_population=None
                )
                t1 = time.perf_counter()

                record["cpu_energy"] = float(E_cpu)
                record["cpu_runtime_s"] = float(t1 - t0)
                record["status_cpu"] = "OK"

                if ref is not None and ref != 0:
                    record["cpu_gap_pct"] = 100.0 * (E_cpu - ref) / ref

                print(f"[CPU] done. E={E_cpu:.0f} time={record['cpu_runtime_s']:.3f}s")

            except Exception as e:
                record["status_cpu"] = "FAIL"
                record["error_cpu"] = repr(e)
                print(f"[CPU] FAILED: {repr(e)}")

            # Write partial record immediately after CPU attempt
            append_jsonl(jsonl_path, record)
            append_csv(csv_path, record, columns)
            print("[LOG] wrote after CPU step")

            # -------------------------
            # GPU seed-only (population init on GPU, solve on CPU)
            # -------------------------
            if not HAS_CUPY:
                record["status_gpu_seedonly"] = "SKIP_NO_CUPY"
                append_jsonl(jsonl_path, record)
                append_csv(csv_path, record, columns)
                print("[GPU-SEEDONLY] skipped (no CuPy).")
                continue

            try:
                # CPU RNG controls algorithmic decisions after initialization
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                # Warm up GPU once per trial
                _ = cp.asarray(np.array([1, -1], dtype=np.int8))
                cp.cuda.Stream.null.synchronize()

                print("[GPU-SEEDONLY] generating initial population on GPU...")
                pop_seed = generate_population_gpu_seed_only(N=N, K=K, seed=seed + 2000*t + N)

                print("[GPU-SEEDONLY] starting CPU solve with GPU-seeded population...")
                g0 = time.perf_counter()
                E_seed, _ = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    stop_at_target=False,
                    init_population=pop_seed
                )
                g1 = time.perf_counter()

                record["gpu_seedonly_energy"] = float(E_seed)
                record["gpu_seedonly_runtime_s"] = float(g1 - g0)
                record["status_gpu_seedonly"] = "OK"

                if ref is not None and ref != 0:
                    record["gpu_seedonly_gap_pct"] = 100.0 * (E_seed - ref) / ref

                print(f"[GPU-SEEDONLY] done. E={E_seed:.0f} time={record['gpu_seedonly_runtime_s']:.3f}s")

            except Exception as e:
                record["status_gpu_seedonly"] = "FAIL"
                record["error_gpu_seedonly"] = repr(e)
                print(f"[GPU-SEEDONLY] FAILED: {repr(e)}")

            # Write record again after GPU-seed-only attempt
            append_jsonl(jsonl_path, record)
            append_csv(csv_path, record, columns)
            print("[LOG] wrote after GPU-seed-only step")

    print("\n[DONE] Benchmark loop finished.")


if __name__ == "__main__":
    # Conservative settings for crash-prone runs:
    # Lower G_max first, then scale up once stable.
    run_benchmark_highN(
        N_list=(20, 25, 30),
        K=100,
        G_max=300,     # start lower to avoid crashing; increase later
        trials=1,
        seed=42,
        out_prefix="highN_mts_cpu_vs_gpu_seedonly"
    )

[LOG] Writing incremental results to:
  highN_mts_cpu_vs_gpu_seedonly.csv
  highN_mts_cpu_vs_gpu_seedonly.jsonl


===== N=20 (ref=26) =====

[RUN] N=20 trial=0 @ 2026-02-01 06:18:33
[CPU] starting...
[CPU] done. E=26 time=8.532s
[LOG] wrote after CPU step
[GPU-SEEDONLY] generating initial population on GPU...
[GPU-SEEDONLY] starting CPU solve with GPU-seeded population...
[GPU-SEEDONLY] done. E=26 time=8.626s
[LOG] wrote after GPU-seed-only step

===== N=25 (ref=36) =====

[RUN] N=25 trial=0 @ 2026-02-01 06:18:51
[CPU] starting...
[CPU] done. E=36 time=15.928s
[LOG] wrote after CPU step
[GPU-SEEDONLY] generating initial population on GPU...
[GPU-SEEDONLY] starting CPU solve with GPU-seeded population...
[GPU-SEEDONLY] done. E=44 time=15.850s
[LOG] wrote after GPU-seed-only step

===== N=30 (ref=59) =====

[RUN] N=30 trial=0 @ 2026-02-01 06:19:22
[CPU] starting...
[CPU] done. E=67 time=28.086s
[LOG] wrote after CPU step
[GPU-SEEDONLY] generating initial population on GPU...
[GPU-SEEDONL

In [14]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# Population generation
# ============================================================

def generate_population_cpu(N: int, K: int) -> list[np.ndarray]:
    """CPU RNG population: list of K arrays shape (N,) with ±1 int8."""
    pop = np.random.choice([-1, 1], size=(K, N)).astype(np.int8)
    return [pop[i].copy() for i in range(K)]


def generate_population_gpu_seedonly(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    GPU RNG population on GPU, then transfer ONCE to CPU.
    Returns list of numpy arrays shape (N,), dtype int8.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run GPU-seeded mode.")

    cp.random.seed(seed)
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}
    pop_cpu = cp.asnumpy(pop_gpu)
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Tabu + MTS (CPU)
#   - same structure as your code, but with:
#       * aspiration fix (best admissible tabu move competes)
#       * optional timeout checks per generation
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_cpu(s0: np.ndarray, max_iter: int = None) -> np.ndarray:
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration: tabu moves only allowed if they beat global best,
            # but must still be best among admissible candidates
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)
            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int,
    G_max: int,
    p_comb: float = 0.9,
    p_mut: float | None = None,
    init_population: list[np.ndarray] | None = None,
    timeout_s: float | None = None,
    print_every: int = 25,
):
    """
    Full CPU MTS.
    Returns: (best_energy, best_sequence, generations_completed, status)
      status ∈ {"OK", "TIMEOUT"}
    """
    if p_mut is None:
        p_mut = 1.0 / N

    if init_population is None:
        population = generate_population_cpu(N, K)
    else:
        population = init_population

    # initial best
    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    start = time.perf_counter()

    for g in range(1, G_max + 1):
        if timeout_s is not None and (time.perf_counter() - start) > timeout_s:
            return float(E_best), s_best, g - 1, "TIMEOUT"

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        # mutation
        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        # local search
        child = tabu_search_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

        if print_every and (g % print_every == 0):
            elapsed = time.perf_counter() - start
            print(f"    [gen {g}/{G_max}] best E={E_best:.0f} elapsed={elapsed:.1f}s")

    return float(E_best), s_best, G_max, "OK"


# ============================================================
# CSV logging (incremental, crash-safe)
# ============================================================

def append_csv(path: str, record: dict, header: list[str]):
    new_file = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header) + "\n")
        f.write(",".join(str(record.get(k, "")) for k in header) + "\n")


# ============================================================
# Benchmark runner
# ============================================================

def run_full_benchmark(
    N_list=(20, 25, 30, 35, 40, 45),
    K=100,
    G_max=200,
    trials=1,
    seed=42,
    timeout_s=None,          # e.g. 600 for 10 minutes per run
    out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv",
):
    header = [
        "timestamp",
        "N",
        "trial",
        "K",
        "G_max",
        "timeout_s",
        "reference_energy",

        "cpu_seed_energy",
        "cpu_seed_runtime_s",
        "cpu_seed_gens",
        "cpu_seed_status",
        "cpu_seed_gap_pct",
        "cpu_seed_error",

        "gpu_seed_energy",
        "gpu_seed_runtime_s",
        "gpu_seed_gens",
        "gpu_seed_status",
        "gpu_seed_gap_pct",
        "gpu_seed_error",
    ]

    print(f"[LOG] Writing CSV incrementally to: {out_csv}")
    if not HAS_CUPY:
        print("[WARN] CuPy not available: GPU-seeded runs will be skipped.")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, "")
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            base_record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "K": K,
                "G_max": G_max,
                "timeout_s": timeout_s if timeout_s is not None else "",
                "reference_energy": ref,
                "cpu_seed_energy": "",
                "cpu_seed_runtime_s": "",
                "cpu_seed_gens": "",
                "cpu_seed_status": "NOT_RUN",
                "cpu_seed_gap_pct": "",
                "cpu_seed_error": "",
                "gpu_seed_energy": "",
                "gpu_seed_runtime_s": "",
                "gpu_seed_gens": "",
                "gpu_seed_status": "NOT_RUN",
                "gpu_seed_gap_pct": "",
                "gpu_seed_error": "",
            }

            # -------------------------
            # CPU-seeded full run
            # -------------------------
            record = dict(base_record)
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)

                print("[CPU-SEEDED] generating population on CPU + solving...")
                t0 = time.perf_counter()
                E_cpu, _, gens_cpu, status_cpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=None,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                t1 = time.perf_counter()

                record["cpu_seed_energy"] = float(E_cpu)
                record["cpu_seed_runtime_s"] = float(t1 - t0)
                record["cpu_seed_gens"] = gens_cpu
                record["cpu_seed_status"] = status_cpu

                if ref != "" and ref != 0:
                    record["cpu_seed_gap_pct"] = 100.0 * (E_cpu - float(ref)) / float(ref)

                print(f"[CPU-SEEDED] done. E={E_cpu:.0f} time={record['cpu_seed_runtime_s']:.3f}s status={status_cpu}")

            except Exception as e:
                record["cpu_seed_status"] = "FAIL"
                record["cpu_seed_error"] = repr(e)
                print(f"[CPU-SEEDED] FAILED: {repr(e)}")

            # Log after CPU step
            append_csv(out_csv, record, header)
            print("[LOG] wrote after CPU-seeded run")

            # -------------------------
            # GPU-seeded full run (seed-only GPU)
            # -------------------------
            record2 = dict(record)  # start from CPU record, then fill GPU columns
            if not HAS_CUPY:
                record2["gpu_seed_status"] = "SKIP_NO_CUPY"
                append_csv(out_csv, record2, header)
                print("[GPU-SEEDED] skipped (no CuPy).")
                continue

            try:
                # IMPORTANT: reseed CPU RNG for the solver decisions
                # (keeps GPU-seeded runs reproducible across repeated experiments)
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                print("[GPU-SEEDED] generating population on GPU...")
                pop_gpu_seed = generate_population_gpu_seedonly(N=N, K=K, seed=seed + 3000*t + N)

                print("[GPU-SEEDED] solving on CPU with GPU-seeded population...")
                g0 = time.perf_counter()
                E_gpu, _, gens_gpu, status_gpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=pop_gpu_seed,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                g1 = time.perf_counter()

                record2["gpu_seed_energy"] = float(E_gpu)
                record2["gpu_seed_runtime_s"] = float(g1 - g0)
                record2["gpu_seed_gens"] = gens_gpu
                record2["gpu_seed_status"] = status_gpu

                if ref != "" and ref != 0:
                    record2["gpu_seed_gap_pct"] = 100.0 * (E_gpu - float(ref)) / float(ref)

                print(f"[GPU-SEEDED] done. E={E_gpu:.0f} time={record2['gpu_seed_runtime_s']:.3f}s status={status_gpu}")

            except Exception as e:
                record2["gpu_seed_status"] = "FAIL"
                record2["gpu_seed_error"] = repr(e)
                print(f"[GPU-SEEDED] FAILED: {repr(e)}")

            # Log after GPU step
            append_csv(out_csv, record2, header)
            print("[LOG] wrote after GPU-seeded run")

    print("\n[DONE] All requested N finished (or timed out).")


if __name__ == "__main__":
    # Suggested start: keep G_max modest; scale up once stable.
    run_full_benchmark(
        N_list=(20, 25, 30, 35, 40, 45),
        K=100,
        G_max=200,
        trials=1,
        seed=42,
        timeout_s=900,  # 15 minutes per run; set None to disable
        out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv"
    )


[LOG] Writing CSV incrementally to: mts_fullrun_cpu_vs_gpu_seedonly.csv

===== N=20 (ref=26) =====

[RUN] N=20 trial=0 @ 2026-02-01 06:31:08
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/200] best E=34 elapsed=1.5s
    [gen 100/200] best E=26 elapsed=3.0s
    [gen 150/200] best E=26 elapsed=4.5s
    [gen 200/200] best E=26 elapsed=5.9s
[CPU-SEEDED] done. E=26 time=5.909s status=OK
[LOG] wrote after CPU-seeded run
[GPU-SEEDED] generating population on GPU...
[GPU-SEEDED] solving on CPU with GPU-seeded population...
    [gen 50/200] best E=26 elapsed=1.5s
    [gen 100/200] best E=26 elapsed=3.0s
    [gen 150/200] best E=26 elapsed=4.5s
    [gen 200/200] best E=26 elapsed=5.9s
[GPU-SEEDED] done. E=26 time=5.891s status=OK
[LOG] wrote after GPU-seeded run

===== N=25 (ref=36) =====

[RUN] N=25 trial=0 @ 2026-02-01 06:31:20
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/200] best E=44 elapsed=2.6s
    [gen 100/200] best E=44 elapsed=5.4s
    [gen 150

In [15]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# Population generation
# ============================================================

def generate_population_cpu(N: int, K: int) -> list[np.ndarray]:
    """CPU RNG population: list of K arrays shape (N,) with ±1 int8."""
    pop = np.random.choice([-1, 1], size=(K, N)).astype(np.int8)
    return [pop[i].copy() for i in range(K)]


def generate_population_gpu_seedonly(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    GPU RNG population on GPU, then transfer ONCE to CPU.
    Returns list of numpy arrays shape (N,), dtype int8.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run GPU-seeded mode.")

    cp.random.seed(seed)
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}
    pop_cpu = cp.asnumpy(pop_gpu)
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Tabu + MTS (CPU)
#   - same structure as your code, but with:
#       * aspiration fix (best admissible tabu move competes)
#       * optional timeout checks per generation
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_cpu(s0: np.ndarray, max_iter: int = None) -> np.ndarray:
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration: tabu moves only allowed if they beat global best,
            # but must still be best among admissible candidates
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)
            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int,
    G_max: int,
    p_comb: float = 0.9,
    p_mut: float | None = None,
    init_population: list[np.ndarray] | None = None,
    timeout_s: float | None = None,
    print_every: int = 25,
):
    """
    Full CPU MTS.
    Returns: (best_energy, best_sequence, generations_completed, status)
      status ∈ {"OK", "TIMEOUT"}
    """
    if p_mut is None:
        p_mut = 1.0 / N

    if init_population is None:
        population = generate_population_cpu(N, K)
    else:
        population = init_population

    # initial best
    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    start = time.perf_counter()

    for g in range(1, G_max + 1):
        if timeout_s is not None and (time.perf_counter() - start) > timeout_s:
            return float(E_best), s_best, g - 1, "TIMEOUT"

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        # mutation
        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        # local search
        child = tabu_search_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

        if print_every and (g % print_every == 0):
            elapsed = time.perf_counter() - start
            print(f"    [gen {g}/{G_max}] best E={E_best:.0f} elapsed={elapsed:.1f}s")

    return float(E_best), s_best, G_max, "OK"


# ============================================================
# CSV logging (incremental, crash-safe)
# ============================================================

def append_csv(path: str, record: dict, header: list[str]):
    new_file = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header) + "\n")
        f.write(",".join(str(record.get(k, "")) for k in header) + "\n")


# ============================================================
# Benchmark runner
# ============================================================

def run_full_benchmark(
    N_list=(20, 25, 30, 35, 40, 45, 50),
    K=100,
    G_max=200,
    trials=1,
    seed=42,
    timeout_s=None,          # e.g. 600 for 10 minutes per run
    out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv",
):
    header = [
        "timestamp",
        "N",
        "trial",
        "K",
        "G_max",
        "timeout_s",
        "reference_energy",

        "cpu_seed_energy",
        "cpu_seed_runtime_s",
        "cpu_seed_gens",
        "cpu_seed_status",
        "cpu_seed_gap_pct",
        "cpu_seed_error",

        "gpu_seed_energy",
        "gpu_seed_runtime_s",
        "gpu_seed_gens",
        "gpu_seed_status",
        "gpu_seed_gap_pct",
        "gpu_seed_error",
    ]

    print(f"[LOG] Writing CSV incrementally to: {out_csv}")
    if not HAS_CUPY:
        print("[WARN] CuPy not available: GPU-seeded runs will be skipped.")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, "")
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            base_record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "K": K,
                "G_max": G_max,
                "timeout_s": timeout_s if timeout_s is not None else "",
                "reference_energy": ref,
                "cpu_seed_energy": "",
                "cpu_seed_runtime_s": "",
                "cpu_seed_gens": "",
                "cpu_seed_status": "NOT_RUN",
                "cpu_seed_gap_pct": "",
                "cpu_seed_error": "",
                "gpu_seed_energy": "",
                "gpu_seed_runtime_s": "",
                "gpu_seed_gens": "",
                "gpu_seed_status": "NOT_RUN",
                "gpu_seed_gap_pct": "",
                "gpu_seed_error": "",
            }

            # -------------------------
            # CPU-seeded full run
            # -------------------------
            record = dict(base_record)
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)

                print("[CPU-SEEDED] generating population on CPU + solving...")
                t0 = time.perf_counter()
                E_cpu, _, gens_cpu, status_cpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=None,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                t1 = time.perf_counter()

                record["cpu_seed_energy"] = float(E_cpu)
                record["cpu_seed_runtime_s"] = float(t1 - t0)
                record["cpu_seed_gens"] = gens_cpu
                record["cpu_seed_status"] = status_cpu

                if ref != "" and ref != 0:
                    record["cpu_seed_gap_pct"] = 100.0 * (E_cpu - float(ref)) / float(ref)

                print(f"[CPU-SEEDED] done. E={E_cpu:.0f} time={record['cpu_seed_runtime_s']:.3f}s status={status_cpu}")

            except Exception as e:
                record["cpu_seed_status"] = "FAIL"
                record["cpu_seed_error"] = repr(e)
                print(f"[CPU-SEEDED] FAILED: {repr(e)}")

            # Log after CPU step
            append_csv(out_csv, record, header)
            print("[LOG] wrote after CPU-seeded run")

            # -------------------------
            # GPU-seeded full run (seed-only GPU)
            # -------------------------
            record2 = dict(record)  # start from CPU record, then fill GPU columns
            if not HAS_CUPY:
                record2["gpu_seed_status"] = "SKIP_NO_CUPY"
                append_csv(out_csv, record2, header)
                print("[GPU-SEEDED] skipped (no CuPy).")
                continue

            try:
                # IMPORTANT: reseed CPU RNG for the solver decisions
                # (keeps GPU-seeded runs reproducible across repeated experiments)
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                print("[GPU-SEEDED] generating population on GPU...")
                pop_gpu_seed = generate_population_gpu_seedonly(N=N, K=K, seed=seed + 3000*t + N)

                print("[GPU-SEEDED] solving on CPU with GPU-seeded population...")
                g0 = time.perf_counter()
                E_gpu, _, gens_gpu, status_gpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=pop_gpu_seed,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                g1 = time.perf_counter()

                record2["gpu_seed_energy"] = float(E_gpu)
                record2["gpu_seed_runtime_s"] = float(g1 - g0)
                record2["gpu_seed_gens"] = gens_gpu
                record2["gpu_seed_status"] = status_gpu

                if ref != "" and ref != 0:
                    record2["gpu_seed_gap_pct"] = 100.0 * (E_gpu - float(ref)) / float(ref)

                print(f"[GPU-SEEDED] done. E={E_gpu:.0f} time={record2['gpu_seed_runtime_s']:.3f}s status={status_gpu}")

            except Exception as e:
                record2["gpu_seed_status"] = "FAIL"
                record2["gpu_seed_error"] = repr(e)
                print(f"[GPU-SEEDED] FAILED: {repr(e)}")

            # Log after GPU step
            append_csv(out_csv, record2, header)
            print("[LOG] wrote after GPU-seeded run")

    print("\n[DONE] All requested N finished (or timed out).")


if __name__ == "__main__":
    # Suggested start: keep G_max modest; scale up once stable.
    run_full_benchmark(
        N_list=(20, 25, 30, 35, 40, 45, 50),
        K=100,
        G_max=1000,
        trials=1,
        seed=42,
        timeout_s=900,  # 15 minutes per run; set None to disable
        out_csv="mts_fullrun_cpu_vs_gpu_seedonly_02.csv"
    )


[LOG] Writing CSV incrementally to: mts_fullrun_cpu_vs_gpu_seedonly_02.csv

===== N=20 (ref=26) =====

[RUN] N=20 trial=0 @ 2026-02-01 06:38:53
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/1000] best E=34 elapsed=1.5s
    [gen 100/1000] best E=26 elapsed=3.0s
    [gen 150/1000] best E=26 elapsed=4.5s
    [gen 200/1000] best E=26 elapsed=5.9s
    [gen 250/1000] best E=26 elapsed=7.3s
    [gen 300/1000] best E=26 elapsed=8.5s
    [gen 350/1000] best E=26 elapsed=9.8s
    [gen 400/1000] best E=26 elapsed=11.4s
    [gen 450/1000] best E=26 elapsed=12.6s
    [gen 500/1000] best E=26 elapsed=14.0s
    [gen 550/1000] best E=26 elapsed=15.4s
    [gen 600/1000] best E=26 elapsed=16.9s
    [gen 650/1000] best E=26 elapsed=18.4s
    [gen 700/1000] best E=26 elapsed=20.0s
    [gen 750/1000] best E=26 elapsed=21.2s
    [gen 800/1000] best E=26 elapsed=22.8s
    [gen 850/1000] best E=26 elapsed=24.2s
    [gen 900/1000] best E=26 elapsed=25.5s
    [gen 950/1000] best E=26 elapse

KeyboardInterrupt: 

In [16]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# Population generation
# ============================================================

def generate_population_cpu(N: int, K: int) -> list[np.ndarray]:
    """CPU RNG population: list of K arrays shape (N,) with ±1 int8."""
    pop = np.random.choice([-1, 1], size=(K, N)).astype(np.int8)
    return [pop[i].copy() for i in range(K)]


def generate_population_gpu_seedonly(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    GPU RNG population on GPU, then transfer ONCE to CPU.
    Returns list of numpy arrays shape (N,), dtype int8.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run GPU-seeded mode.")

    cp.random.seed(seed)
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}
    pop_cpu = cp.asnumpy(pop_gpu)
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Tabu + MTS (CPU)
#   - same structure as your code, but with:
#       * aspiration fix (best admissible tabu move competes)
#       * optional timeout checks per generation
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_cpu(s0: np.ndarray, max_iter: int = None) -> np.ndarray:
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration: tabu moves only allowed if they beat global best,
            # but must still be best among admissible candidates
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)
            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int,
    G_max: int,
    p_comb: float = 0.9,
    p_mut: float | None = None,
    init_population: list[np.ndarray] | None = None,
    timeout_s: float | None = None,
    print_every: int = 25,
):
    """
    Full CPU MTS.
    Returns: (best_energy, best_sequence, generations_completed, status)
      status ∈ {"OK", "TIMEOUT"}
    """
    if p_mut is None:
        p_mut = 1.0 / N

    if init_population is None:
        population = generate_population_cpu(N, K)
    else:
        population = init_population

    # initial best
    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    start = time.perf_counter()

    for g in range(1, G_max + 1):
        if timeout_s is not None and (time.perf_counter() - start) > timeout_s:
            return float(E_best), s_best, g - 1, "TIMEOUT"

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        # mutation
        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        # local search
        child = tabu_search_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

        if print_every and (g % print_every == 0):
            elapsed = time.perf_counter() - start
            print(f"    [gen {g}/{G_max}] best E={E_best:.0f} elapsed={elapsed:.1f}s")

    return float(E_best), s_best, G_max, "OK"


# ============================================================
# CSV logging (incremental, crash-safe)
# ============================================================

def append_csv(path: str, record: dict, header: list[str]):
    new_file = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header) + "\n")
        f.write(",".join(str(record.get(k, "")) for k in header) + "\n")


# ============================================================
# Benchmark runner
# ============================================================

def run_full_benchmark(
    N_list=(20, 25, 30, 35, 40, 45, 50),
    K=100,
    G_max=200,
    trials=1,
    seed=42,
    timeout_s=None,          # e.g. 600 for 10 minutes per run
    out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv",
):
    header = [
        "timestamp",
        "N",
        "trial",
        "K",
        "G_max",
        "timeout_s",
        "reference_energy",

        "cpu_seed_energy",
        "cpu_seed_runtime_s",
        "cpu_seed_gens",
        "cpu_seed_status",
        "cpu_seed_gap_pct",
        "cpu_seed_error",

        "gpu_seed_energy",
        "gpu_seed_runtime_s",
        "gpu_seed_gens",
        "gpu_seed_status",
        "gpu_seed_gap_pct",
        "gpu_seed_error",
    ]

    print(f"[LOG] Writing CSV incrementally to: {out_csv}")
    if not HAS_CUPY:
        print("[WARN] CuPy not available: GPU-seeded runs will be skipped.")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, "")
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            base_record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "K": K,
                "G_max": G_max,
                "timeout_s": timeout_s if timeout_s is not None else "",
                "reference_energy": ref,
                "cpu_seed_energy": "",
                "cpu_seed_runtime_s": "",
                "cpu_seed_gens": "",
                "cpu_seed_status": "NOT_RUN",
                "cpu_seed_gap_pct": "",
                "cpu_seed_error": "",
                "gpu_seed_energy": "",
                "gpu_seed_runtime_s": "",
                "gpu_seed_gens": "",
                "gpu_seed_status": "NOT_RUN",
                "gpu_seed_gap_pct": "",
                "gpu_seed_error": "",
            }

            # -------------------------
            # CPU-seeded full run
            # -------------------------
            record = dict(base_record)
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)

                print("[CPU-SEEDED] generating population on CPU + solving...")
                t0 = time.perf_counter()
                E_cpu, _, gens_cpu, status_cpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=None,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                t1 = time.perf_counter()

                record["cpu_seed_energy"] = float(E_cpu)
                record["cpu_seed_runtime_s"] = float(t1 - t0)
                record["cpu_seed_gens"] = gens_cpu
                record["cpu_seed_status"] = status_cpu

                if ref != "" and ref != 0:
                    record["cpu_seed_gap_pct"] = 100.0 * (E_cpu - float(ref)) / float(ref)

                print(f"[CPU-SEEDED] done. E={E_cpu:.0f} time={record['cpu_seed_runtime_s']:.3f}s status={status_cpu}")

            except Exception as e:
                record["cpu_seed_status"] = "FAIL"
                record["cpu_seed_error"] = repr(e)
                print(f"[CPU-SEEDED] FAILED: {repr(e)}")

            # Log after CPU step
            append_csv(out_csv, record, header)
            print("[LOG] wrote after CPU-seeded run")

            # -------------------------
            # GPU-seeded full run (seed-only GPU)
            # -------------------------
            record2 = dict(record)  # start from CPU record, then fill GPU columns
            if not HAS_CUPY:
                record2["gpu_seed_status"] = "SKIP_NO_CUPY"
                append_csv(out_csv, record2, header)
                print("[GPU-SEEDED] skipped (no CuPy).")
                continue

            try:
                # IMPORTANT: reseed CPU RNG for the solver decisions
                # (keeps GPU-seeded runs reproducible across repeated experiments)
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                print("[GPU-SEEDED] generating population on GPU...")
                pop_gpu_seed = generate_population_gpu_seedonly(N=N, K=K, seed=seed + 3000*t + N)

                print("[GPU-SEEDED] solving on CPU with GPU-seeded population...")
                g0 = time.perf_counter()
                E_gpu, _, gens_gpu, status_gpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=pop_gpu_seed,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                g1 = time.perf_counter()

                record2["gpu_seed_energy"] = float(E_gpu)
                record2["gpu_seed_runtime_s"] = float(g1 - g0)
                record2["gpu_seed_gens"] = gens_gpu
                record2["gpu_seed_status"] = status_gpu

                if ref != "" and ref != 0:
                    record2["gpu_seed_gap_pct"] = 100.0 * (E_gpu - float(ref)) / float(ref)

                print(f"[GPU-SEEDED] done. E={E_gpu:.0f} time={record2['gpu_seed_runtime_s']:.3f}s status={status_gpu}")

            except Exception as e:
                record2["gpu_seed_status"] = "FAIL"
                record2["gpu_seed_error"] = repr(e)
                print(f"[GPU-SEEDED] FAILED: {repr(e)}")

            # Log after GPU step
            append_csv(out_csv, record2, header)
            print("[LOG] wrote after GPU-seeded run")

    print("\n[DONE] All requested N finished (or timed out).")


if __name__ == "__main__":
    # Suggested start: keep G_max modest; scale up once stable.
    run_full_benchmark(
        N_list=(20, 22, 24, 26, 28, 30, 32, 34),
        K=100,
        G_max=300,
        trials=1,
        seed=42,
        timeout_s=900,  # 15 minutes per run; set None to disable
        out_csv="mts_fullrun_cpu_vs_gpu_seedonly_03.csv"
    )


[LOG] Writing CSV incrementally to: mts_fullrun_cpu_vs_gpu_seedonly_03.csv

===== N=20 (ref=26) =====

[RUN] N=20 trial=0 @ 2026-02-01 07:00:38
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/300] best E=34 elapsed=1.5s
    [gen 100/300] best E=26 elapsed=3.0s
    [gen 150/300] best E=26 elapsed=4.5s
    [gen 200/300] best E=26 elapsed=5.9s
    [gen 250/300] best E=26 elapsed=7.3s
    [gen 300/300] best E=26 elapsed=8.5s
[CPU-SEEDED] done. E=26 time=8.519s status=OK
[LOG] wrote after CPU-seeded run
[GPU-SEEDED] generating population on GPU...
[GPU-SEEDED] solving on CPU with GPU-seeded population...
    [gen 50/300] best E=26 elapsed=1.5s
    [gen 100/300] best E=26 elapsed=3.0s
    [gen 150/300] best E=26 elapsed=4.5s
    [gen 200/300] best E=26 elapsed=5.9s
    [gen 250/300] best E=26 elapsed=7.3s
    [gen 300/300] best E=26 elapsed=8.6s
[GPU-SEEDED] done. E=26 time=8.574s status=OK
[LOG] wrote after GPU-seeded run

===== N=22 (ref=39) =====

[RUN] N=22 trial=0 @ 2

In [1]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# Population generation
# ============================================================

def generate_population_cpu(N: int, K: int) -> list[np.ndarray]:
    """CPU RNG population: list of K arrays shape (N,) with ±1 int8."""
    pop = np.random.choice([-1, 1], size=(K, N)).astype(np.int8)
    return [pop[i].copy() for i in range(K)]


def generate_population_gpu_seedonly(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    GPU RNG population on GPU, then transfer ONCE to CPU.
    Returns list of numpy arrays shape (N,), dtype int8.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run GPU-seeded mode.")

    cp.random.seed(seed)
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}
    pop_cpu = cp.asnumpy(pop_gpu)
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Tabu + MTS (CPU)
#   - same structure as your code, but with:
#       * aspiration fix (best admissible tabu move competes)
#       * optional timeout checks per generation
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_cpu(s0: np.ndarray, max_iter: int = None) -> np.ndarray:
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration: tabu moves only allowed if they beat global best,
            # but must still be best among admissible candidates
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)
            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int,
    G_max: int,
    p_comb: float = 0.9,
    p_mut: float | None = None,
    init_population: list[np.ndarray] | None = None,
    timeout_s: float | None = None,
    print_every: int = 25,
):
    """
    Full CPU MTS.
    Returns: (best_energy, best_sequence, generations_completed, status)
      status ∈ {"OK", "TIMEOUT"}
    """
    if p_mut is None:
        p_mut = 1.0 / N

    if init_population is None:
        population = generate_population_cpu(N, K)
    else:
        population = init_population

    # initial best
    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    start = time.perf_counter()

    for g in range(1, G_max + 1):
        if timeout_s is not None and (time.perf_counter() - start) > timeout_s:
            return float(E_best), s_best, g - 1, "TIMEOUT"

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        # mutation
        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        # local search
        child = tabu_search_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

        if print_every and (g % print_every == 0):
            elapsed = time.perf_counter() - start
            print(f"    [gen {g}/{G_max}] best E={E_best:.0f} elapsed={elapsed:.1f}s")

    return float(E_best), s_best, G_max, "OK"


# ============================================================
# CSV logging (incremental, crash-safe)
# ============================================================

def append_csv(path: str, record: dict, header: list[str]):
    new_file = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header) + "\n")
        f.write(",".join(str(record.get(k, "")) for k in header) + "\n")


# ============================================================
# Benchmark runner
# ============================================================

def run_full_benchmark(
    N_list=(20, 25, 30, 35, 40, 45, 50),
    K=100,
    G_max=200,
    trials=1,
    seed=42,
    timeout_s=None,          # e.g. 600 for 10 minutes per run
    out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv",
):
    header = [
        "timestamp",
        "N",
        "trial",
        "K",
        "G_max",
        "timeout_s",
        "reference_energy",

        "cpu_seed_energy",
        "cpu_seed_runtime_s",
        "cpu_seed_gens",
        "cpu_seed_status",
        "cpu_seed_gap_pct",
        "cpu_seed_error",

        "gpu_seed_energy",
        "gpu_seed_runtime_s",
        "gpu_seed_gens",
        "gpu_seed_status",
        "gpu_seed_gap_pct",
        "gpu_seed_error",
    ]

    print(f"[LOG] Writing CSV incrementally to: {out_csv}")
    if not HAS_CUPY:
        print("[WARN] CuPy not available: GPU-seeded runs will be skipped.")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, "")
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            base_record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "K": K,
                "G_max": G_max,
                "timeout_s": timeout_s if timeout_s is not None else "",
                "reference_energy": ref,
                "cpu_seed_energy": "",
                "cpu_seed_runtime_s": "",
                "cpu_seed_gens": "",
                "cpu_seed_status": "NOT_RUN",
                "cpu_seed_gap_pct": "",
                "cpu_seed_error": "",
                "gpu_seed_energy": "",
                "gpu_seed_runtime_s": "",
                "gpu_seed_gens": "",
                "gpu_seed_status": "NOT_RUN",
                "gpu_seed_gap_pct": "",
                "gpu_seed_error": "",
            }

            # -------------------------
            # CPU-seeded full run
            # -------------------------
            record = dict(base_record)
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)

                print("[CPU-SEEDED] generating population on CPU + solving...")
                t0 = time.perf_counter()
                E_cpu, _, gens_cpu, status_cpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=None,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                t1 = time.perf_counter()

                record["cpu_seed_energy"] = float(E_cpu)
                record["cpu_seed_runtime_s"] = float(t1 - t0)
                record["cpu_seed_gens"] = gens_cpu
                record["cpu_seed_status"] = status_cpu

                if ref != "" and ref != 0:
                    record["cpu_seed_gap_pct"] = 100.0 * (E_cpu - float(ref)) / float(ref)

                print(f"[CPU-SEEDED] done. E={E_cpu:.0f} time={record['cpu_seed_runtime_s']:.3f}s status={status_cpu}")

            except Exception as e:
                record["cpu_seed_status"] = "FAIL"
                record["cpu_seed_error"] = repr(e)
                print(f"[CPU-SEEDED] FAILED: {repr(e)}")

            # Log after CPU step
            append_csv(out_csv, record, header)
            print("[LOG] wrote after CPU-seeded run")

            # -------------------------
            # GPU-seeded full run (seed-only GPU)
            # -------------------------
            record2 = dict(record)  # start from CPU record, then fill GPU columns
            if not HAS_CUPY:
                record2["gpu_seed_status"] = "SKIP_NO_CUPY"
                append_csv(out_csv, record2, header)
                print("[GPU-SEEDED] skipped (no CuPy).")
                continue

            try:
                # IMPORTANT: reseed CPU RNG for the solver decisions
                # (keeps GPU-seeded runs reproducible across repeated experiments)
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                print("[GPU-SEEDED] generating population on GPU...")
                pop_gpu_seed = generate_population_gpu_seedonly(N=N, K=K, seed=seed + 3000*t + N)

                print("[GPU-SEEDED] solving on CPU with GPU-seeded population...")
                g0 = time.perf_counter()
                E_gpu, _, gens_gpu, status_gpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=pop_gpu_seed,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                g1 = time.perf_counter()

                record2["gpu_seed_energy"] = float(E_gpu)
                record2["gpu_seed_runtime_s"] = float(g1 - g0)
                record2["gpu_seed_gens"] = gens_gpu
                record2["gpu_seed_status"] = status_gpu

                if ref != "" and ref != 0:
                    record2["gpu_seed_gap_pct"] = 100.0 * (E_gpu - float(ref)) / float(ref)

                print(f"[GPU-SEEDED] done. E={E_gpu:.0f} time={record2['gpu_seed_runtime_s']:.3f}s status={status_gpu}")

            except Exception as e:
                record2["gpu_seed_status"] = "FAIL"
                record2["gpu_seed_error"] = repr(e)
                print(f"[GPU-SEEDED] FAILED: {repr(e)}")

            # Log after GPU step
            append_csv(out_csv, record2, header)
            print("[LOG] wrote after GPU-seeded run")

    print("\n[DONE] All requested N finished (or timed out).")


if __name__ == "__main__":
    # Suggested start: keep G_max modest; scale up once stable.
    run_full_benchmark(
        N_list=(20, 22, 24, 26, 28, 30, 32, 34),
        K=100,
        G_max=200,
        trials=1,
        seed=42,
        timeout_s=900,  # 15 minutes per run; set None to disable
        out_csv="mts_fullrun_cpu_vs_gpu_seedonly_04.csv"
    )

[LOG] Writing CSV incrementally to: mts_fullrun_cpu_vs_gpu_seedonly_04.csv

===== N=20 (ref=26) =====

[RUN] N=20 trial=0 @ 2026-02-01 07:31:45
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/200] best E=34 elapsed=1.5s
    [gen 100/200] best E=26 elapsed=3.0s
    [gen 150/200] best E=26 elapsed=4.5s
    [gen 200/200] best E=26 elapsed=5.9s
[CPU-SEEDED] done. E=26 time=5.921s status=OK
[LOG] wrote after CPU-seeded run
[GPU-SEEDED] generating population on GPU...
[GPU-SEEDED] solving on CPU with GPU-seeded population...
    [gen 50/200] best E=26 elapsed=1.5s
    [gen 100/200] best E=26 elapsed=3.1s
    [gen 150/200] best E=26 elapsed=4.5s
    [gen 200/200] best E=26 elapsed=6.0s
[GPU-SEEDED] done. E=26 time=5.976s status=OK
[LOG] wrote after GPU-seeded run

===== N=22 (ref=39) =====

[RUN] N=22 trial=0 @ 2026-02-01 07:32:02
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/200] best E=39 elapsed=2.0s
    [gen 100/200] best E=39 elapsed=4.0s
    [gen 

In [ ]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# Population generation
# ============================================================

def generate_population_cpu(N: int, K: int) -> list[np.ndarray]:
    """CPU RNG population: list of K arrays shape (N,) with ±1 int8."""
    pop = np.random.choice([-1, 1], size=(K, N)).astype(np.int8)
    return [pop[i].copy() for i in range(K)]


def generate_population_gpu_seedonly(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    GPU RNG population on GPU, then transfer ONCE to CPU.
    Returns list of numpy arrays shape (N,), dtype int8.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run GPU-seeded mode.")

    cp.random.seed(seed)
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}
    pop_cpu = cp.asnumpy(pop_gpu)
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Tabu + MTS (CPU)
#   - same structure as your code, but with:
#       * aspiration fix (best admissible tabu move competes)
#       * optional timeout checks per generation
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_cpu(s0: np.ndarray, max_iter: int = None) -> np.ndarray:
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration: tabu moves only allowed if they beat global best,
            # but must still be best among admissible candidates
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)
            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int,
    G_max: int,
    p_comb: float = 0.9,
    p_mut: float | None = None,
    init_population: list[np.ndarray] | None = None,
    timeout_s: float | None = None,
    print_every: int = 25,
):
    """
    Full CPU MTS.
    Returns: (best_energy, best_sequence, generations_completed, status)
      status ∈ {"OK", "TIMEOUT"}
    """
    if p_mut is None:
        p_mut = 1.0 / N

    if init_population is None:
        population = generate_population_cpu(N, K)
    else:
        population = init_population

    # initial best
    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    start = time.perf_counter()

    for g in range(1, G_max + 1):
        if timeout_s is not None and (time.perf_counter() - start) > timeout_s:
            return float(E_best), s_best, g - 1, "TIMEOUT"

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        # mutation
        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        # local search
        child = tabu_search_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

        if print_every and (g % print_every == 0):
            elapsed = time.perf_counter() - start
            print(f"    [gen {g}/{G_max}] best E={E_best:.0f} elapsed={elapsed:.1f}s")

    return float(E_best), s_best, G_max, "OK"


# ============================================================
# CSV logging (incremental, crash-safe)
# ============================================================

def append_csv(path: str, record: dict, header: list[str]):
    new_file = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header) + "\n")
        f.write(",".join(str(record.get(k, "")) for k in header) + "\n")


# ============================================================
# Benchmark runner
# ============================================================

def run_full_benchmark(
    N_list=(35, 38, 41, 44),
    K=100,
    G_max=200,
    trials=1,
    seed=42,
    timeout_s=None,          # e.g. 600 for 10 minutes per run
    out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv",
):
    header = [
        "timestamp",
        "N",
        "trial",
        "K",
        "G_max",
        "timeout_s",
        "reference_energy",

        "cpu_seed_energy",
        "cpu_seed_runtime_s",
        "cpu_seed_gens",
        "cpu_seed_status",
        "cpu_seed_gap_pct",
        "cpu_seed_error",

        "gpu_seed_energy",
        "gpu_seed_runtime_s",
        "gpu_seed_gens",
        "gpu_seed_status",
        "gpu_seed_gap_pct",
        "gpu_seed_error",
    ]

    print(f"[LOG] Writing CSV incrementally to: {out_csv}")
    if not HAS_CUPY:
        print("[WARN] CuPy not available: GPU-seeded runs will be skipped.")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, "")
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            base_record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "K": K,
                "G_max": G_max,
                "timeout_s": timeout_s if timeout_s is not None else "",
                "reference_energy": ref,
                "cpu_seed_energy": "",
                "cpu_seed_runtime_s": "",
                "cpu_seed_gens": "",
                "cpu_seed_status": "NOT_RUN",
                "cpu_seed_gap_pct": "",
                "cpu_seed_error": "",
                "gpu_seed_energy": "",
                "gpu_seed_runtime_s": "",
                "gpu_seed_gens": "",
                "gpu_seed_status": "NOT_RUN",
                "gpu_seed_gap_pct": "",
                "gpu_seed_error": "",
            }

            # -------------------------
            # CPU-seeded full run
            # -------------------------
            record = dict(base_record)
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)

                print("[CPU-SEEDED] generating population on CPU + solving...")
                t0 = time.perf_counter()
                E_cpu, _, gens_cpu, status_cpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=None,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                t1 = time.perf_counter()

                record["cpu_seed_energy"] = float(E_cpu)
                record["cpu_seed_runtime_s"] = float(t1 - t0)
                record["cpu_seed_gens"] = gens_cpu
                record["cpu_seed_status"] = status_cpu

                if ref != "" and ref != 0:
                    record["cpu_seed_gap_pct"] = 100.0 * (E_cpu - float(ref)) / float(ref)

                print(f"[CPU-SEEDED] done. E={E_cpu:.0f} time={record['cpu_seed_runtime_s']:.3f}s status={status_cpu}")

            except Exception as e:
                record["cpu_seed_status"] = "FAIL"
                record["cpu_seed_error"] = repr(e)
                print(f"[CPU-SEEDED] FAILED: {repr(e)}")

            # Log after CPU step
            append_csv(out_csv, record, header)
            print("[LOG] wrote after CPU-seeded run")

            # -------------------------
            # GPU-seeded full run (seed-only GPU)
            # -------------------------
            record2 = dict(record)  # start from CPU record, then fill GPU columns
            if not HAS_CUPY:
                record2["gpu_seed_status"] = "SKIP_NO_CUPY"
                append_csv(out_csv, record2, header)
                print("[GPU-SEEDED] skipped (no CuPy).")
                continue

            try:
                # IMPORTANT: reseed CPU RNG for the solver decisions
                # (keeps GPU-seeded runs reproducible across repeated experiments)
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                print("[GPU-SEEDED] generating population on GPU...")
                pop_gpu_seed = generate_population_gpu_seedonly(N=N, K=K, seed=seed + 3000*t + N)

                print("[GPU-SEEDED] solving on CPU with GPU-seeded population...")
                g0 = time.perf_counter()
                E_gpu, _, gens_gpu, status_gpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=pop_gpu_seed,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                g1 = time.perf_counter()

                record2["gpu_seed_energy"] = float(E_gpu)
                record2["gpu_seed_runtime_s"] = float(g1 - g0)
                record2["gpu_seed_gens"] = gens_gpu
                record2["gpu_seed_status"] = status_gpu

                if ref != "" and ref != 0:
                    record2["gpu_seed_gap_pct"] = 100.0 * (E_gpu - float(ref)) / float(ref)

                print(f"[GPU-SEEDED] done. E={E_gpu:.0f} time={record2['gpu_seed_runtime_s']:.3f}s status={status_gpu}")

            except Exception as e:
                record2["gpu_seed_status"] = "FAIL"
                record2["gpu_seed_error"] = repr(e)
                print(f"[GPU-SEEDED] FAILED: {repr(e)}")

            # Log after GPU step
            append_csv(out_csv, record2, header)
            print("[LOG] wrote after GPU-seeded run")

    print("\n[DONE] All requested N finished (or timed out).")


if __name__ == "__main__":
    # Suggested start: keep G_max modest; scale up once stable.
    run_full_benchmark(
        N_list=(35, 38, 41, 44),
        K=100,
        G_max=1000,
        trials=1,
        seed=42,
        timeout_s=900,  # 15 minutes per run; set None to disable
        out_csv="mts_fullrun_cpu_vs_gpu_seedonly_05.csv"
    )

[LOG] Writing CSV incrementally to: mts_fullrun_cpu_vs_gpu_seedonly_05.csv

===== N=35 (ref=73) =====

[RUN] N=35 trial=0 @ 2026-02-01 07:36:15
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/1000] best E=97 elapsed=6.7s
    [gen 100/1000] best E=97 elapsed=14.5s
    [gen 150/1000] best E=97 elapsed=21.9s
    [gen 200/1000] best E=97 elapsed=29.7s
    [gen 250/1000] best E=97 elapsed=36.9s
    [gen 300/1000] best E=89 elapsed=44.4s
    [gen 350/1000] best E=89 elapsed=52.0s
    [gen 400/1000] best E=89 elapsed=59.2s
    [gen 450/1000] best E=89 elapsed=66.9s
    [gen 500/1000] best E=89 elapsed=75.1s
    [gen 550/1000] best E=89 elapsed=82.9s
    [gen 600/1000] best E=89 elapsed=91.6s
    [gen 650/1000] best E=89 elapsed=98.9s
    [gen 700/1000] best E=89 elapsed=106.5s
    [gen 750/1000] best E=89 elapsed=114.8s
    [gen 800/1000] best E=89 elapsed=122.8s
    [gen 850/1000] best E=89 elapsed=130.5s
    [gen 900/1000] best E=89 elapsed=139.0s
    [gen 950/1000] best 

In [2]:
import os
import time
import json
import random
import numpy as np

# -----------------------------
# Optional GPU backend (CuPy)
# -----------------------------
try:
    import cupy as cp
    HAS_CUPY = True
except Exception:
    cp = None
    HAS_CUPY = False


# -----------------------------
# LABS reference table
# -----------------------------
LABS_OPTIMAL = {
    2: 1, 3: 1, 4: 2, 5: 2, 6: 7, 7: 3, 8: 8, 9: 12, 10: 13,
    11: 5, 12: 10, 13: 6, 14: 19, 15: 15, 16: 24, 17: 32, 18: 25, 19: 29, 20: 26,
    21: 26, 22: 39, 23: 47, 24: 36, 25: 36, 26: 45, 27: 37, 28: 50, 29: 62, 30: 59,
    31: 67, 32: 64, 33: 64, 34: 65, 35: 73, 36: 82, 37: 86, 38: 87, 39: 99, 40: 108,
    41: 108, 42: 101, 43: 109, 44: 122, 45: 118, 46: 131, 47: 135, 48: 140, 49: 136, 50: 153,
    51: 153, 52: 166, 53: 170, 54: 175, 55: 171, 56: 192, 57: 188, 58: 197, 59: 205, 60: 218,
    61: 226, 62: 235, 63: 207, 64: 208, 65: 240, 66: 257
}


# ============================================================
# Energy (CPU)
# ============================================================

def energy_numpy(s: np.ndarray) -> float:
    N = len(s)
    e = 0
    for k in range(1, N):
        Ck = int(np.sum(s[:N-k] * s[k:]))
        e += Ck * Ck
    return float(e)


# ============================================================
# Population generation
# ============================================================

def generate_population_cpu(N: int, K: int) -> list[np.ndarray]:
    """CPU RNG population: list of K arrays shape (N,) with ±1 int8."""
    pop = np.random.choice([-1, 1], size=(K, N)).astype(np.int8)
    return [pop[i].copy() for i in range(K)]


def generate_population_gpu_seedonly(N: int, K: int, seed: int) -> list[np.ndarray]:
    """
    GPU RNG population on GPU, then transfer ONCE to CPU.
    Returns list of numpy arrays shape (N,), dtype int8.
    """
    if not HAS_CUPY:
        raise RuntimeError("CuPy not available; cannot run GPU-seeded mode.")

    cp.random.seed(seed)
    pop_gpu = cp.random.randint(0, 2, size=(K, N), dtype=cp.int8)
    pop_gpu = pop_gpu * 2 - 1  # {0,1} -> {-1,+1}
    pop_cpu = cp.asnumpy(pop_gpu)
    return [pop_cpu[i].copy() for i in range(K)]


# ============================================================
# Tabu + MTS (CPU)
#   - same structure as your code, but with:
#       * aspiration fix (best admissible tabu move competes)
#       * optional timeout checks per generation
# ============================================================

def flip_inplace(arr, i):
    arr[i] *= -1


def tabu_search_cpu(s0: np.ndarray, max_iter: int = None) -> np.ndarray:
    N = int(s0.shape[0])
    s = s0.copy()
    s_best = s.copy()
    E_best = energy_numpy(s_best)

    tabu_list = np.zeros(N, dtype=np.int32)

    if max_iter is None:
        M = random.randint(max(1, N // 10), max(2, 3 * N // 2))
    else:
        M = int(max_iter)

    for t in range(1, M + 1):
        best_i = -1
        best_neighbor_energy = float("inf")

        for i in range(N):
            flip_inplace(s, i)
            E_neighbor = energy_numpy(s)
            flip_inplace(s, i)

            is_tabu = tabu_list[i] > t

            # aspiration: tabu moves only allowed if they beat global best,
            # but must still be best among admissible candidates
            if is_tabu:
                if E_neighbor < E_best and E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i
            else:
                if E_neighbor < best_neighbor_energy:
                    best_neighbor_energy = E_neighbor
                    best_i = i

        if best_i >= 0:
            flip_inplace(s, best_i)
            tenure = random.randint(max(1, M // 10), max(2, M // 2))
            tabu_list[best_i] = t + tenure

            if best_neighbor_energy < E_best:
                s_best = s.copy()
                E_best = best_neighbor_energy

    return s_best


def memetic_tabu_search_cpu(
    N: int,
    K: int,
    G_max: int,
    p_comb: float = 0.9,
    p_mut: float | None = None,
    init_population: list[np.ndarray] | None = None,
    timeout_s: float | None = None,
    print_every: int = 25,
):
    """
    Full CPU MTS.
    Returns: (best_energy, best_sequence, generations_completed, status)
      status ∈ {"OK", "TIMEOUT"}
    """
    if p_mut is None:
        p_mut = 1.0 / N

    if init_population is None:
        population = generate_population_cpu(N, K)
    else:
        population = init_population

    # initial best
    s_best = population[0].copy()
    E_best = energy_numpy(s_best)
    for ind in population[1:]:
        E = energy_numpy(ind)
        if E < E_best:
            s_best = ind.copy()
            E_best = E

    start = time.perf_counter()

    for g in range(1, G_max + 1):
        if timeout_s is not None and (time.perf_counter() - start) > timeout_s:
            return float(E_best), s_best, g - 1, "TIMEOUT"

        if random.random() < p_comb:
            p1 = random.choice(population)
            p2 = random.choice(population)
            cut = random.randint(1, N - 1)
            child = np.concatenate([p1[:cut], p2[cut:]]).copy()
        else:
            child = random.choice(population).copy()

        # mutation
        for i in range(N):
            if random.random() < p_mut:
                child[i] *= -1

        # local search
        child = tabu_search_cpu(child)
        E_child = energy_numpy(child)

        if E_child < E_best:
            s_best = child.copy()
            E_best = E_child

        population[random.randint(0, K - 1)] = child

        if print_every and (g % print_every == 0):
            elapsed = time.perf_counter() - start
            print(f"    [gen {g}/{G_max}] best E={E_best:.0f} elapsed={elapsed:.1f}s")

    return float(E_best), s_best, G_max, "OK"


# ============================================================
# CSV logging (incremental, crash-safe)
# ============================================================

def append_csv(path: str, record: dict, header: list[str]):
    new_file = not os.path.exists(path)
    with open(path, "a", encoding="utf-8") as f:
        if new_file:
            f.write(",".join(header) + "\n")
        f.write(",".join(str(record.get(k, "")) for k in header) + "\n")


# ============================================================
# Benchmark runner
# ============================================================

def run_full_benchmark(
    N_list=(5, 10, 15, 20, 25, 30, 35, 40, 45),
    K=100,
    G_max=200,
    trials=1,
    seed=42,
    timeout_s=None,          # e.g. 600 for 10 minutes per run
    out_csv="mts_fullrun_cpu_vs_gpu_seedonly.csv",
):
    header = [
        "timestamp",
        "N",
        "trial",
        "K",
        "G_max",
        "timeout_s",
        "reference_energy",

        "cpu_seed_energy",
        "cpu_seed_runtime_s",
        "cpu_seed_gens",
        "cpu_seed_status",
        "cpu_seed_gap_pct",
        "cpu_seed_error",

        "gpu_seed_energy",
        "gpu_seed_runtime_s",
        "gpu_seed_gens",
        "gpu_seed_status",
        "gpu_seed_gap_pct",
        "gpu_seed_error",
    ]

    print(f"[LOG] Writing CSV incrementally to: {out_csv}")
    if not HAS_CUPY:
        print("[WARN] CuPy not available: GPU-seeded runs will be skipped.")

    for N in N_list:
        ref = LABS_OPTIMAL.get(N, "")
        print(f"\n===== N={N} (ref={ref}) =====")

        for t in range(trials):
            stamp = time.strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n[RUN] N={N} trial={t} @ {stamp}")

            base_record = {
                "timestamp": stamp,
                "N": N,
                "trial": t,
                "K": K,
                "G_max": G_max,
                "timeout_s": timeout_s if timeout_s is not None else "",
                "reference_energy": ref,
                "cpu_seed_energy": "",
                "cpu_seed_runtime_s": "",
                "cpu_seed_gens": "",
                "cpu_seed_status": "NOT_RUN",
                "cpu_seed_gap_pct": "",
                "cpu_seed_error": "",
                "gpu_seed_energy": "",
                "gpu_seed_runtime_s": "",
                "gpu_seed_gens": "",
                "gpu_seed_status": "NOT_RUN",
                "gpu_seed_gap_pct": "",
                "gpu_seed_error": "",
            }

            # -------------------------
            # CPU-seeded full run
            # -------------------------
            record = dict(base_record)
            try:
                np.random.seed(seed + 1000*t + N)
                random.seed(seed + 1000*t + N)

                print("[CPU-SEEDED] generating population on CPU + solving...")
                t0 = time.perf_counter()
                E_cpu, _, gens_cpu, status_cpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=None,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                t1 = time.perf_counter()

                record["cpu_seed_energy"] = float(E_cpu)
                record["cpu_seed_runtime_s"] = float(t1 - t0)
                record["cpu_seed_gens"] = gens_cpu
                record["cpu_seed_status"] = status_cpu

                if ref != "" and ref != 0:
                    record["cpu_seed_gap_pct"] = 100.0 * (E_cpu - float(ref)) / float(ref)

                print(f"[CPU-SEEDED] done. E={E_cpu:.0f} time={record['cpu_seed_runtime_s']:.3f}s status={status_cpu}")

            except Exception as e:
                record["cpu_seed_status"] = "FAIL"
                record["cpu_seed_error"] = repr(e)
                print(f"[CPU-SEEDED] FAILED: {repr(e)}")

            # Log after CPU step
            append_csv(out_csv, record, header)
            print("[LOG] wrote after CPU-seeded run")

            # -------------------------
            # GPU-seeded full run (seed-only GPU)
            # -------------------------
            record2 = dict(record)  # start from CPU record, then fill GPU columns
            if not HAS_CUPY:
                record2["gpu_seed_status"] = "SKIP_NO_CUPY"
                append_csv(out_csv, record2, header)
                print("[GPU-SEEDED] skipped (no CuPy).")
                continue

            try:
                # IMPORTANT: reseed CPU RNG for the solver decisions
                # (keeps GPU-seeded runs reproducible across repeated experiments)
                np.random.seed(seed + 2000*t + N)
                random.seed(seed + 2000*t + N)

                print("[GPU-SEEDED] generating population on GPU...")
                pop_gpu_seed = generate_population_gpu_seedonly(N=N, K=K, seed=seed + 3000*t + N)

                print("[GPU-SEEDED] solving on CPU with GPU-seeded population...")
                g0 = time.perf_counter()
                E_gpu, _, gens_gpu, status_gpu = memetic_tabu_search_cpu(
                    N=N, K=K, G_max=G_max,
                    init_population=pop_gpu_seed,
                    timeout_s=timeout_s,
                    print_every=50,
                )
                g1 = time.perf_counter()

                record2["gpu_seed_energy"] = float(E_gpu)
                record2["gpu_seed_runtime_s"] = float(g1 - g0)
                record2["gpu_seed_gens"] = gens_gpu
                record2["gpu_seed_status"] = status_gpu

                if ref != "" and ref != 0:
                    record2["gpu_seed_gap_pct"] = 100.0 * (E_gpu - float(ref)) / float(ref)

                print(f"[GPU-SEEDED] done. E={E_gpu:.0f} time={record2['gpu_seed_runtime_s']:.3f}s status={status_gpu}")

            except Exception as e:
                record2["gpu_seed_status"] = "FAIL"
                record2["gpu_seed_error"] = repr(e)
                print(f"[GPU-SEEDED] FAILED: {repr(e)}")

            # Log after GPU step
            append_csv(out_csv, record2, header)
            print("[LOG] wrote after GPU-seeded run")

    print("\n[DONE] All requested N finished (or timed out).")


if __name__ == "__main__":
    # Suggested start: keep G_max modest; scale up once stable.
    run_full_benchmark(
        N_list=(5, 10, 15, 20, 25, 30, 35, 40, 45),
        K=100,
        G_max=200,
        trials=1,
        seed=42,
        timeout_s=900,  # 15 minutes per run; set None to disable
        out_csv="mts_fullrun_cpu_vs_gpu_seedonly_05.csv"
    )

[LOG] Writing CSV incrementally to: mts_fullrun_cpu_vs_gpu_seedonly_05.csv

===== N=5 (ref=2) =====

[RUN] N=5 trial=0 @ 2026-02-01 13:56:12
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/200] best E=2 elapsed=0.0s
    [gen 100/200] best E=2 elapsed=0.1s
    [gen 150/200] best E=2 elapsed=0.1s
    [gen 200/200] best E=2 elapsed=0.1s
[CPU-SEEDED] done. E=2 time=0.128s status=OK
[LOG] wrote after CPU-seeded run
[GPU-SEEDED] generating population on GPU...
[GPU-SEEDED] solving on CPU with GPU-seeded population...
    [gen 50/200] best E=2 elapsed=0.0s
    [gen 100/200] best E=2 elapsed=0.0s
    [gen 150/200] best E=2 elapsed=0.1s
    [gen 200/200] best E=2 elapsed=0.1s
[GPU-SEEDED] done. E=2 time=0.093s status=OK
[LOG] wrote after GPU-seeded run

===== N=10 (ref=13) =====

[RUN] N=10 trial=0 @ 2026-02-01 13:56:12
[CPU-SEEDED] generating population on CPU + solving...
    [gen 50/200] best E=13 elapsed=0.1s
    [gen 100/200] best E=13 elapsed=0.3s
    [gen 150/200] best